In [1]:
import os
os.chdir('../..')

In [2]:
import numpy as np
import pandas as pd

import ctgan
from ctgan import CTGANSynthesizer

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
dataset = 'Chicago'

In [4]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [5]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,3.93477,2,3,3,detached,6,0,30,1,4,PTE,20.166667
1,drive,2,SHOPPING,0.31557,3,3,3,detached,7,0,54,1,5,FTE,17.500000
2,drive,2,SHOPPING,0.28349,1,1,0,detached,3,0,80,1,3,PTE,9.333333
3,drive,2,OTHER,0.69417,2,2,0,detached,5,1,42,1,5,FTE,13.783333
4,passenger,1,SHOPPING,4.30666,2,2,1,detached,4,0,32,0,3,Unemployed,11.566667


In [6]:
if dataset is 'Chicago':
    discrete_columns = [
        'choice',
        'travel_dow',
        'trip_purpose',
        'hh_vehicles',
        'hh_size',
        'hh_bikes',
        'hh_descr',
        'hh_income',
        'gender',
        'license',
        'education_level',
        'work_status'
    ]
elif dataset is 'LPMC':
    discrete_columns = [
        'travel_mode',
        'purpose',
        'fueltype',
        'faretype',
        'bus_scale',
        'survey_year',
        'travel_year',
        'travel_month',
        'travel_date',
        'day_of_week',
        'female',
        'driving_license',
        'car_ownership',
        'pt_n_interchanges',
        'cost_driving_con_charge'
    ]

In [7]:
output_folder = '../output/' + dataset + '/CTGAN/'

# Train

In [8]:
ctgan = CTGANSynthesizer(verbose=True, cuda=True, batch_size=500)

In [9]:
%%time
ctgan.fit(df, discrete_columns, epochs=1000)

D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sklearn\mixture\_base.py:269: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


Epoch 1, Loss G:  1.7106,Loss D: -0.4491
Epoch 2, Loss G:  1.1326,Loss D: -0.0852
Epoch 3, Loss G:  0.8236,Loss D:  0.0227
Epoch 4, Loss G:  0.9873,Loss D:  0.1244
Epoch 5, Loss G:  0.9938,Loss D:  0.0196
Epoch 6, Loss G:  0.8368,Loss D:  0.0018
Epoch 7, Loss G:  0.8826,Loss D: -0.0690
Epoch 8, Loss G:  0.7938,Loss D:  0.0137
Epoch 9, Loss G:  0.7936,Loss D: -0.0679
Epoch 10, Loss G:  0.6183,Loss D: -0.0173
Epoch 11, Loss G:  0.5654,Loss D: -0.0580
Epoch 12, Loss G:  0.4273,Loss D:  0.0883
Epoch 13, Loss G:  0.4489,Loss D:  0.0301
Epoch 14, Loss G:  0.3530,Loss D: -0.0021
Epoch 15, Loss G:  0.1621,Loss D:  0.0441
Epoch 16, Loss G:  0.2969,Loss D: -0.1108
Epoch 17, Loss G:  0.0533,Loss D:  0.0562
Epoch 18, Loss G:  0.1880,Loss D: -0.0207
Epoch 19, Loss G: -0.1661,Loss D:  0.0211
Epoch 20, Loss G: -0.1124,Loss D: -0.1804
Epoch 21, Loss G: -0.2071,Loss D:  0.1388
Epoch 22, Loss G: -0.3756,Loss D: -0.0311
Epoch 23, Loss G: -0.3761,Loss D: -0.0586
Epoch 24, Loss G: -0.4168,Loss D: -0.0485
E

Epoch 195, Loss G: -0.6182,Loss D:  0.1222
Epoch 196, Loss G: -0.5793,Loss D: -0.1916
Epoch 197, Loss G: -0.4754,Loss D:  0.0469
Epoch 198, Loss G: -0.8767,Loss D: -0.2368
Epoch 199, Loss G: -0.8331,Loss D: -0.4253
Epoch 200, Loss G: -0.7526,Loss D:  0.0032
Epoch 201, Loss G: -0.9548,Loss D: -0.0508
Epoch 202, Loss G: -1.0512,Loss D: -0.1565
Epoch 203, Loss G: -0.8345,Loss D: -0.0116
Epoch 204, Loss G: -0.5268,Loss D:  0.0465
Epoch 205, Loss G: -0.4661,Loss D: -0.0823
Epoch 206, Loss G: -0.8494,Loss D:  0.0436
Epoch 207, Loss G: -1.0326,Loss D:  0.0119
Epoch 208, Loss G: -0.7627,Loss D:  0.2193
Epoch 209, Loss G: -0.9518,Loss D: -0.0967
Epoch 210, Loss G: -0.9747,Loss D: -0.1383
Epoch 211, Loss G: -1.2964,Loss D:  0.0061
Epoch 212, Loss G: -1.1293,Loss D: -0.0308
Epoch 213, Loss G: -1.2285,Loss D: -0.3379
Epoch 214, Loss G: -0.8014,Loss D: -0.0429
Epoch 215, Loss G: -1.1524,Loss D:  0.2409
Epoch 216, Loss G: -0.8428,Loss D:  0.1024
Epoch 217, Loss G: -0.9114,Loss D: -0.2015
Epoch 218, 

Epoch 386, Loss G: -1.1077,Loss D: -0.0194
Epoch 387, Loss G: -0.9421,Loss D: -0.2253
Epoch 388, Loss G: -1.0049,Loss D: -0.3170
Epoch 389, Loss G: -0.9776,Loss D:  0.0934
Epoch 390, Loss G: -0.8737,Loss D: -0.0264
Epoch 391, Loss G: -1.1031,Loss D: -0.1398
Epoch 392, Loss G: -1.1771,Loss D:  0.0382
Epoch 393, Loss G: -1.2612,Loss D: -0.1354
Epoch 394, Loss G: -1.2759,Loss D:  0.0984
Epoch 395, Loss G: -0.9739,Loss D:  0.1026
Epoch 396, Loss G: -0.8473,Loss D:  0.0843
Epoch 397, Loss G: -1.2848,Loss D:  0.1390
Epoch 398, Loss G: -0.9783,Loss D: -0.0247
Epoch 399, Loss G: -1.1840,Loss D: -0.2993
Epoch 400, Loss G: -1.2360,Loss D:  0.1269
Epoch 401, Loss G: -1.0458,Loss D: -0.3226
Epoch 402, Loss G: -1.4122,Loss D:  0.1003
Epoch 403, Loss G: -1.1658,Loss D:  0.1267
Epoch 404, Loss G: -1.1793,Loss D: -0.2634
Epoch 405, Loss G: -1.1312,Loss D: -0.0568
Epoch 406, Loss G: -1.2575,Loss D: -0.0868
Epoch 407, Loss G: -1.2462,Loss D:  0.1350
Epoch 408, Loss G: -1.2855,Loss D:  0.1178
Epoch 409, 

Epoch 577, Loss G: -0.6959,Loss D:  0.0384
Epoch 578, Loss G: -0.7470,Loss D: -0.0068
Epoch 579, Loss G: -0.6599,Loss D:  0.0938
Epoch 580, Loss G: -0.7343,Loss D:  0.2095
Epoch 581, Loss G: -0.6759,Loss D: -0.2643
Epoch 582, Loss G: -0.7185,Loss D: -0.1454
Epoch 583, Loss G: -0.7753,Loss D: -0.2972
Epoch 584, Loss G: -0.6775,Loss D: -0.1719
Epoch 585, Loss G: -0.6803,Loss D: -0.0258
Epoch 586, Loss G: -0.4093,Loss D: -0.0688
Epoch 587, Loss G: -0.6418,Loss D:  0.1312
Epoch 588, Loss G: -0.7087,Loss D: -0.1366
Epoch 589, Loss G: -0.5831,Loss D:  0.0458
Epoch 590, Loss G: -0.9809,Loss D: -0.0111
Epoch 591, Loss G: -0.8647,Loss D:  0.0781
Epoch 592, Loss G: -0.6297,Loss D: -0.1498
Epoch 593, Loss G: -0.5177,Loss D: -0.0003
Epoch 594, Loss G: -0.8794,Loss D:  0.0148
Epoch 595, Loss G: -0.5140,Loss D: -0.1012
Epoch 596, Loss G: -0.6064,Loss D: -0.2534
Epoch 597, Loss G: -0.7355,Loss D: -0.3513
Epoch 598, Loss G: -0.6906,Loss D: -0.0831
Epoch 599, Loss G: -0.7183,Loss D:  0.1184
Epoch 600, 

Epoch 768, Loss G: -0.7085,Loss D:  0.1543
Epoch 769, Loss G: -0.6388,Loss D: -0.0695
Epoch 770, Loss G: -0.8773,Loss D: -0.1946
Epoch 771, Loss G: -0.6586,Loss D:  0.0626
Epoch 772, Loss G: -0.5175,Loss D: -0.2256
Epoch 773, Loss G: -0.8443,Loss D: -0.1296
Epoch 774, Loss G: -0.6700,Loss D:  0.0693
Epoch 775, Loss G: -0.8204,Loss D:  0.0750
Epoch 776, Loss G: -0.6883,Loss D: -0.0266
Epoch 777, Loss G: -0.8823,Loss D: -0.0127
Epoch 778, Loss G: -0.6443,Loss D: -0.1214
Epoch 779, Loss G: -0.5713,Loss D: -0.0693
Epoch 780, Loss G: -0.6566,Loss D: -0.0736
Epoch 781, Loss G: -0.5815,Loss D: -0.2373
Epoch 782, Loss G: -0.7682,Loss D:  0.4461
Epoch 783, Loss G: -0.9001,Loss D: -0.0793
Epoch 784, Loss G: -0.6937,Loss D: -0.1968
Epoch 785, Loss G: -0.7038,Loss D: -0.0586
Epoch 786, Loss G: -0.6348,Loss D: -0.0425
Epoch 787, Loss G: -0.7741,Loss D:  0.0924
Epoch 788, Loss G: -0.8608,Loss D:  0.1227
Epoch 789, Loss G: -1.2876,Loss D:  0.0047
Epoch 790, Loss G: -1.1228,Loss D: -0.0101
Epoch 791, 

Epoch 959, Loss G: -1.0650,Loss D: -0.1295
Epoch 960, Loss G: -1.0062,Loss D:  0.0947
Epoch 961, Loss G: -0.8254,Loss D: -0.3014
Epoch 962, Loss G: -0.9839,Loss D:  0.1955
Epoch 963, Loss G: -0.7913,Loss D:  0.0978
Epoch 964, Loss G: -0.8883,Loss D: -0.1831
Epoch 965, Loss G: -1.1874,Loss D: -0.1912
Epoch 966, Loss G: -1.2684,Loss D: -0.2500
Epoch 967, Loss G: -1.5281,Loss D: -0.0451
Epoch 968, Loss G: -1.1028,Loss D: -0.0722
Epoch 969, Loss G: -0.8787,Loss D:  0.1010
Epoch 970, Loss G: -0.8333,Loss D:  0.0484
Epoch 971, Loss G: -0.9467,Loss D: -0.1290
Epoch 972, Loss G: -1.0436,Loss D: -0.1978
Epoch 973, Loss G: -1.1014,Loss D: -0.0542
Epoch 974, Loss G: -0.9574,Loss D: -0.1723
Epoch 975, Loss G: -1.1253,Loss D: -0.1772
Epoch 976, Loss G: -1.0274,Loss D: -0.1818
Epoch 977, Loss G: -0.8541,Loss D: -0.0369
Epoch 978, Loss G: -0.9807,Loss D: -0.0027
Epoch 979, Loss G: -1.0477,Loss D:  0.1745
Epoch 980, Loss G: -0.7736,Loss D:  0.0452
Epoch 981, Loss G: -0.8210,Loss D:  0.1545
Epoch 982, 

In [11]:
ctgan.save(output_folder + 'trained.pkl')